# OCT simulations with Python

This notebook describes basic OCT signal processing steps and visualizes them. The work is mostly based on the work presented in

_J. Kalkman, Fourier-Domain Optical Coherence Tomography Signal Analysis and Numerical Modeling, International Journal of Optics 2017_

In [2]:
# basic functions
import numpy as np
import matplotlib.pyplot as plt

# for enabling interaction with the plots
from ipywidgets import interact, interactive, fixed, interact_manual

### Basic Fourier-domain OCT simulation

We consider a Fourier domain OCT system. The signal is acquired in the $k$-domain and Fourier transformed to the $z$-domain.

In [3]:
# OCT parameters
lambdac=950         # center wavelength [nm]
dlambda=150         # bandwidth [nm]
P0=1                # input power [mW]
tau=1e-6            # integration time [s]
N=1024              # number of pixels
alpha=0.5           # intensity splitting ratio

kc=2*np.pi/(lambdac*1e-9)
dk=1e9*dlambda*2*np.pi/lambdac**2             # FWHM_k
kmin=kc-2*dk                        # 2*dk is necessary to avoid problems in the z-domain
kmax=kc+2*dk

k=np.linspace(kmin,kmax,N)
deltak=k[1]-k[0]
n0=np.linspace(0, N-1, N)       # pixel axis
z=np.linspace(-0.5*np.pi/deltak, 0.5*np.pi/deltak, N)
deltaz=z[2]-z[1]

In [17]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import scipy.interpolate

def makeplot(P0, dlambda, zmir, chirp, dispersion):
    # P0 in mW
    # dlambda in nm
    # zmir in mm 

    dk=1e9*dlambda*2*np.pi/lambdac**2
    sigmak = dk/(2*np.sqrt(2*np.log(2))) 

    # # here a non-linear chirp is created
    chirp*=1e-7*chirp
    kchirp=k[0]+(1-chirp*(N-1)*deltak)*n0*deltak+chirp*n0**2*deltak**2
    
    # here the dispersion is added to the signal
    dispersion=1e-4*dispersion
        
    # Calculation of the OCT signal
    Sks=(P0*1e-3)/(np.sqrt(2*np.pi*sigmak**2))*np.exp(-(k-kc)**2/(2*sigmak**2))

    Ukref=np.sqrt(alpha*(1-alpha))*np.sqrt(Sks)
    Uksam=np.sqrt(alpha*(1-alpha))*np.sqrt(Sks)*np.exp(1j*2*k*zmir*1e-3)*np.exp(1j*dispersion*(k-kc)**2/kc)
    Uksamchirp=np.sqrt(alpha*(1-alpha))*np.sqrt(Sks)*np.exp(1j*2*kchirp*zmir*1e-3)
    Ikint=np.real(Uksam+Ukref*np.conj(Uksam+Ukref))-np.real(Ukref*np.conj(Ukref))-np.real(Uksam*np.conj(Uksam))
    Ikintchirp=np.real(Uksamchirp+Ukref*np.conj(Uksamchirp+Ukref))-np.real(Ukref*np.conj(Ukref))-np.real(Uksam*np.conj(Uksamchirp))
    
    fig, axes = plt.subplots(1,3)
    plt.figure(fig).set_figwidth(15)
    plt.subplot(131)
    axes[0].plot(k, Ikint, '-.b', label='ideal OCT signal')
    axes[0].plot(k, Ikintchirp, '-.r', label='chirped OCT signal')
    plt.title('OCT spectrum')
    plt.xlabel('Wavenumber (rad/m)')
    plt.ylabel('Intensity (arb. units)')
    plt.grid(), plt.legend(fontsize="6")

    plt.subplot(132)
    plt.plot(k, k, '-b', label='lineak k')
    plt.plot(k, kchirp, '-r', label='chirped k')
    plt.title('OCT spectral phase')
    plt.xlabel('Wavenumber (rad/m)')
    plt.ylabel('Phase (rad)')
    plt.grid(), plt.legend(fontsize="6")

    iz=np.fft.fftshift(np.fft.ifft(Ikint))
    izchirp=np.fft.fftshift(np.fft.ifft(Ikintchirp))
    #f = scipy.interpolate.interp1d(kchirp, Ikintchirp, kind='cubic')
    ##klin=np.linspace(np.amin(kchirp), np.amax(kchirp), N)
    #Ikintcorr = f(klin)
    #izdechirp=np.fft.fftshift(np.fft.ifft(Ikintcorr))

    plt.subplot(133) 
    axes[2].plot(1e3*z, np.abs(iz)**2, '-b', label='ideal OCT signal')
    axes[2].plot(1e3*z, np.abs(izchirp)**2, '--r', label='chirped OCT signal')
    plt.title('OCT scan')
    plt.xlabel('Depth (mm)')
    plt.ylabel('OCT intensity (arb. units)')
    plt.grid(), plt.legend(fontsize="6")

    # switch between +z and -z
    peakpos=int(1e-3*zmir/deltaz)+int(N/2)
    zrange=10
        
    ins = inset_axes(axes[2], width="30%", height=2.0, loc=1)
    plt.plot(1e3*z[int(peakpos-zrange):int(peakpos+zrange)], np.abs(iz[int(peakpos-zrange):int(peakpos+zrange)])**2, '-b', label='ideal OCT signal')
    plt.plot(1e3*z[int(peakpos-zrange):int(peakpos+zrange)], np.abs(izchirp[int(peakpos-zrange):int(peakpos+zrange)])**2, '--r', label='chirped OCT signal')
    plt.grid()

    plt.tight_layout()
    plt.show()

interactive(makeplot, P0=(0, 5, 0.1), dlambda=(0, 400,10), zmir=(1e3*np.min(z), 1e3*np.max(z), 0.1), chirp=(0,1,0.1), dispersion=(0,1,0.1))

interactive(children=(FloatSlider(value=2.0, description='P0', max=5.0), IntSlider(value=200, description='dla…